## Collecting Data 
Source of datasets :

1.   From IEEE : https://ieee-dataport.org/open-access/fruitsgb-top-indian-fruits-quality
2.   From Kaggle : https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification

But, we already combine both and make a drive. Below is the link for zip dataset (google drive): 
https://drive.google.com/file/d/1s0mHiO3wF_shUHO8NwjytBD1svWAFSr_/view?usp=sharing






In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
from google.colab import drive

#to connect drive account to colab
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import shutil
import os

source = r'/content/gdrive/MyDrive/dataset.zip'
destination = r'/content/dataset.zip'

In [8]:
shutil.copyfile(source, destination)

'/content/dataset.zip'

In [9]:
#extract zip file
import zipfile 
local_zip = destination
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()